# BLIP NoCaps Evaluation (Kaggle)

- Enable Internet in the Kaggle notebook, and add the dataset containing `nocap_val_4500_captions.json` and `nocaps_test_image_info.json` to the *Data* tab.
- The notebook uses run_nocaps_eval.py to download images in parallel; the `--workers` option is set automatically based on `cpu_count()` (typically ~4 cores → 8 workers is safe).
- If you already have the images, set `SKIP_DOWNLOAD = True` in the setup cell to skip the download step; the following steps remain unchanged.

In [ ]:
from pathlib import Path
import json

BASE_DIR = Path("/kaggle/working")

DATASET_ROOT = Path("/kaggle/input/nocaps")
VAL_JSON = DATASET_ROOT / "nocaps_val_4500_captions.json"
TEST_JSON = DATASET_ROOT / "nocaps_test_image_info.json"

REPO_URL = "https://github.com/TMG301-Skubidu/Image-Captioning-Vietnamese-BLIP.git"
REPO_DIR = BASE_DIR / "BLIP"
CLI_SCRIPT = REPO_DIR / "tools/run_nocaps_eval.py"

IMAGE_ROOT = BASE_DIR / "nocaps_images"
ANN_ROOT = BASE_DIR / "nocaps_annotations"
OUTPUT_DIR = BASE_DIR / "NoCaps"

SKIP_DOWNLOAD = False  # đặt True nếu đã có đủ ảnh trong IMAGE_ROOT
WORKERS_OVERRIDE = None  # đặt số nguyên để ép số worker, None = tự động

for path in [IMAGE_ROOT / "val", IMAGE_ROOT / "test", ANN_ROOT, OUTPUT_DIR]:
    path.mkdir(parents=True, exist_ok=True)

print(f"VAL_JSON exists: {VAL_JSON.exists()} -> {VAL_JSON}")
print(f"TEST_JSON exists: {TEST_JSON.exists()} -> {TEST_JSON}")


In [ ]:
import subprocess
import sys

if not REPO_DIR.exists():
    subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
else:
    print(f"Repository already available at {REPO_DIR}")

commit = subprocess.check_output(["git", "rev-parse", "HEAD"], cwd=REPO_DIR).decode().strip()
print("Repository commit:", commit)
for rel_path in ["eval_nocaps.py", "data/nocaps_dataset.py", "configs/nocaps.yaml"]:
    path = REPO_DIR / rel_path
    status = "OK" if path.exists() else "MISSING"
    print(f"{rel_path}: {status}")

In [ ]:
import subprocess
import sys

requirements_path = REPO_DIR / "requirements.txt"
subprocess.run([sys.executable, "-m", "pip", "install", "-r", str(requirements_path)], check=True)

In [ ]:
import os
from pathlib import Path
import torch
import timm

os.chdir(REPO_DIR)
print("Working directory:", Path.cwd())
print("Torch:", torch.__version__, "CUDA available:", torch.cuda.is_available())
print("Timm:", timm.__version__)

In [ ]:
def summarize_image(entry):
    keys = ["file_name", "id", "domain", "coco_url", "open_images_id"]
    return {k: entry.get(k) for k in keys if k in entry}

with open(VAL_JSON, "r") as f:
    val_raw = json.load(f)
with open(TEST_JSON, "r") as f:
    test_raw = json.load(f)

val_caps = val_raw.get("annotations", [])
print("Val keys:", list(val_raw.keys()))
print("Val images:", len(val_raw["images"]))
print("Val annotations:", len(val_caps))
print("Val sample image:", summarize_image(val_raw["images"][0]))
print("Val sample captions:", [ann["caption"] for ann in val_caps[:3]])

print("Test keys:", list(test_raw.keys()))
print("Test images:", len(test_raw["images"]))
print("Test has annotations:", "annotations" in test_raw)
print("Test sample image:", summarize_image(test_raw["images"][0]))

In [ ]:
import json
import multiprocessing
import subprocess
import sys

workers_detected = multiprocessing.cpu_count() or 1
workers = WORKERS_OVERRIDE or min(32, max(4, workers_detected))
print(f"Detected CPU cores: {workers_detected} -> using {workers} workers")

cmd = [
    sys.executable,
    str(CLI_SCRIPT),
    "--dataset-root", str(DATASET_ROOT),
    "--image-root", str(IMAGE_ROOT),
    "--ann-root", str(ANN_ROOT),
    "--output-dir", str(OUTPUT_DIR),
    "--batch-size", "4",
    "--workers", str(workers),
    "--skip-eval",
]
if SKIP_DOWNLOAD:
    cmd.append("--no-download")

print("Running:", " ".join(cmd))
subprocess.run(cmd, check=True)

failures_log = OUTPUT_DIR / "download_failures.json"
if failures_log.exists():
    failures = json.loads(failures_log.read_text())
    print("Download failures - val:", len(failures.get("val", [])), "| test:", len(failures.get("test", [])))
else:
    print("No download_failures.json found yet.")


In [ ]:
import json

val_out = ANN_ROOT / "nocaps_val.json"
test_out = ANN_ROOT / "nocaps_test.json"
failures_log = OUTPUT_DIR / "download_failures.json"

if val_out.exists():
    val_entries = json.loads(val_out.read_text())
    print("Val entries:", len(val_entries), "->", val_out)
    if val_entries:
        print("Sample val entry:", val_entries[0])
else:
    print("Val annotations not ready yet (rerun the data download cell).")

if test_out.exists():
    test_entries = json.loads(test_out.read_text())
    print("Test entries:", len(test_entries), "->", test_out)
    if test_entries:
        print("Sample test entry:", test_entries[0])
else:
    print("Test annotations not ready yet (rerun the data download cell).")

if failures_log.exists():
    failures = json.loads(failures_log.read_text())
    print("Download failure samples (val):", failures.get("val", [])[:3])
    print("Download failure samples (test):", failures.get("test", [])[:3])
else:
    print("No download_failures.json found (no errors or download not finished yet).")

In [ ]:
import json
from pathlib import Path

try:
    import ruamel_yaml as yaml
except ModuleNotFoundError:
    from ruamel import yaml  # type: ignore

expected_config = OUTPUT_DIR / "nocaps_eval.yaml"
if expected_config.exists():
    config_path = expected_config
    config = yaml.load(expected_config.read_text(), Loader=yaml.Loader)
else:
    print("WARNING: not found", expected_config, "-> falling back to template.")
    config_path = Path("configs/nocaps.yaml")
    config = yaml.load(config_path.read_text(), Loader=yaml.Loader)
    config["image_root"] = str(IMAGE_ROOT).replace("\\", "/")
    config["ann_root"] = str(ANN_ROOT).replace("\\", "/")
    config["batch_size"] = min(4, config.get("batch_size", 32))
    expected_config = OUTPUT_DIR / "nocaps_eval.yaml"
    expected_config.write_text(yaml.dump(config))
    print("Wrote new config to:", expected_config)

print("Config path:", expected_config)
print(json.dumps(config, indent=2))

In [ ]:
!python eval_nocaps.py --config /kaggle/working/nocaps_eval.yaml --output_dir /kaggle/working/NoCaps --device cuda --distributed False

In [ ]:
result_dir = OUTPUT_DIR / "result"
val_result_path = result_dir / "val.json"
test_result_path = result_dir / "test.json"

with open(val_result_path, "r") as f:
    val_result = json.load(f)
with open(test_result_path, "r") as f:
    test_result = json.load(f)

print("Val captions:", len(val_result))
print("Test captions:", len(test_result))
print("Sample val captions:")
for sample in val_result[:5]:
    print(sample)

print("Artifacts saved under:", OUTPUT_DIR)

## Next steps
- If `download_failures.json` is not empty, consider manually downloading those images and rerunning the conversion cell.
- Compress `nocaps_images`, `nocaps_annotations`, and `NoCaps/result/test.json` into a separate Kaggle dataset so future notebook runs are faster.
- Upload `NoCaps/result/test.json` to the NoCaps server to receive the official score.